In [19]:
import pandas as pd
import numpy as np
import os
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import sys

from sklearn.model_selection import train_test_split
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from scripts.preprocess_init import columns_selection, drop_outliers, annee, cpv_2et3


In [12]:
db_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'datalab.sqlite')
db_path

'/Users/paulcolas/code/RonanB400/decp_ml/data/datalab.sqlite'

In [13]:
conn = sqlite3.connect(db_path)
db = conn.cursor()
query = f"""
SELECT *
FROM "data.gouv.fr.2022.clean"
"""

data = pd.read_sql_query(query, conn)
data.columns

Index(['uid', 'id', 'nature', 'acheteur_id', 'acheteur_nom', 'acheteur_siren',
       'titulaire_id', 'titulaire_typeIdentifiant', 'titulaire_nom',
       'titulaire_siren', 'objet', 'montant', 'codeCPV', 'procedure',
       'dureeMois', 'dateNotification', 'datePublicationDonnees', 'formePrix',
       'attributionAvance', 'offresRecues', 'marcheInnovant', 'ccag',
       'sousTraitanceDeclaree', 'typeGroupementOperateurs', 'tauxAvance',
       'origineUE', 'origineFrance', 'lieuExecution_code',
       'lieuExecution_typeCode', 'idAccordCadre', 'source_open_data'],
      dtype='object')

In [15]:
df = drop_outliers(data)

In [17]:
df = annee(df)

/Users/paulcolas/code/RonanB400/decp_ml/scripts/preprocess_init.py:26: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df['annee'] = pd.to_datetime(df['annee'], errors='ignore')


In [20]:
df = cpv_2et3(df)

In [22]:
df.head()

,uid,id,nature,acheteur_id,acheteur_nom,acheteur_siren,titulaire_id,titulaire_typeIdentifiant,titulaire_nom,titulaire_siren,...,typeGroupementOperateurs,tauxAvance,origineUE,origineFrance,lieuExecution_code,lieuExecution_typeCode,idAccordCadre,source_open_data,annee,codeCPV_2
0,2154005160001320242024-LOT04,20242024-LOT04,Marché,21540051600013,COMMUNE DE BATILLY,215400516,78885470100018,SIRET,ACOMETAL,788854701,...,Pas de groupement,NaN,NaN,NaN,54980,Code postal,None,data.gouv.fr decp-2024.json,2024,45200000
1,243500667002882021M226MO,2021M226MO,Marché,24350066700288,CC VAL D'ILLE-AUBIGNE,243500667,38373211200032,SIRET,UNIVERS,383732112,...,Solidaire,NaN,NaN,NaN,35520,Code postal,None,data.gouv.fr decp-2024.json,2021,71200000
2,249710047000472024SS_PRD_TRV,2024SS_PRD_TRV,Marché,24971004700047,COMMUNAUTE DE COMMUNES DE MARIE GALANTE,249710047,43387249600016,SIRET,COTRAM B.T.P.,433872496,...,Solidaire,NaN,NaN,NaN,97112,Code postal,None,data.gouv.fr decp-2024.json,2024,45200000
3,6254801990001124-0806-L2,24-0806-L2,Marché,62548019900011,"LA MAISON POUR TOUS, SOCIETE ANONYME COOPERATI...",625480199,55204695502544,SIRET,ENGIE ENERGIE SERVICES,552046955,...,Pas de groupement,NaN,NaN,NaN,39,Code département,None,data.gouv.fr decp-2024.json,2024,50000000
4,20002563300013202424011BCR,202424011BCR,Marché,20002563300013,SI DE RESTAURATION COLLECTIVE,200025633,47698032100238,SIRET,POMONA EPISAVEURS,476980321,...,Pas de groupement,0.0,0.0,0.0,45800,Code postal,None,data.gouv.fr decp-2025-04.json,2024,15000000


In [24]:
X = df[['procedure', 'dureeMois','nature', 'formePrix', 'offresRecues', 'ccag',
        'sousTraitanceDeclaree', 'typeGroupementOperateurs', 'tauxAvance',
        'origineFrance', 'idAccordCadre', 'dateNotification', 'marcheInnovant', 'codeCPV']]
y = df[['montant']]

In [ ]:
# pipeline = create_complete_pipeline(
#     cat='pred_montant',
#     min=20000,
#     max=1000000,
#     top_n=40,
#     level=3
# )
# X_processed = pipeline.fit_transform(X_train)

KeyError: "['txAvance_cat'] not in index"